# CCTV 데이터 시각화 


CCTV 데이터 시각화 과제
=================================

![한양여대로고](https://mblogthumb-phinf.pstatic.net/MjAyMTA4MDNfMjc2/MDAxNjI3OTg3Njg3MTU0.qyp_JW_QnELIQZr5wsyDpidkzk8Ll0sPrf4a6M1lmBEg.F5-amMuD1SkqQUoWIGXCD9t08w-uDX3qusrggS6Kbkog.PNG.enochacademy/%ED%95%9C%EC%96%91%EC%97%AC%EC%9E%90%EB%8C%80%ED%95%99%EA%B5%90-%EB%A1%9C%EA%B3%A0.png?type=w800)

<hr>

| 수강과목 | 탐색적데이터시각화 |
|----------|--------------------|
| 담당교수 | 신해웅 교수님      |
| 학과     | 스마트 IT          |
| 학번     | 2005899            |
| 이름     | 김지연             |
| 제출일자 | 2022.06.17         |

목차
1. 라이브러리 임포트
2. 데이터 적재
3. 버블차트 시각화  
    3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의  
    3.2 2015년도 차트 시각화  
    3.3 2020년도 차트 시각화  
    3.4 이동궤적 시각화  
    3.5 시각화 결과 통합  
4. 단계구분도 시각화
    4.1 자치구 위치 시각화  
    4.2 단계구분도를 반환하는 함수 정의  
    4.3 단계구분도 종합  
5. 결론


## 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import altair as alt

## 2. 데이터 적재

- 필요한 데이터는 다음과 같다. 
  - 인구 및 cctv 데이터는 [여기](https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv)에서 직접 읽어서 `df`라는 데이터 프레임으로 적재하라.  
    이 데이터에 포함된 `no` 열은 지도 시각화를 위한 [자치구 경계선 데이터](https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json)에 포함되어 있는 자치구의 고유 식별 번호이다. 
  
  - 자치구청 위치 데이터는 [여기](https://raw.githubusercontent.com/logistex/vd21/main/offices.csv)에서 직접 읽어서 `offices`라는 데이터 프레임으로 적재하라. 
  - 자치구 경계선에 관한 지도 데이터는 [여기](https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json)에서 제공되는데, 이에 관한 코드는 (여러분의 수고를 줄여주기 위하여) 문제에 포함시켜 두었다.  
  - 일부 csv 파일은 인코딩 방식을 지정해야만 한글을 제대로 읽을 수 있다. 
  - 데이터 파일은 모두 앞서 제공한 URL 주소로부터 직접 읽어 들이라. 
  - 데이터 프레임으로 적재한 직후에는 임의로 5행을 출력하라. 

In [ ]:
# cctv 및 인구 데이터 적재 및 확인
df = pd.read_csv("https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv", engine='python')
df.sample(5)

,no,id,자치구,CCTV_규모_2015,CCTV_규모_2020,인구_규모_2015,인구_규모_2020
5,2,6,동대문구,1436,2315,373824,357014
0,6,1,종로구,1175,1729,163822,158996
24,12,25,강동구,725,2547,463321,463998
4,13,5,광진구,477,2556,375180,360109
18,23,19,영등포구,1278,3773,417811,407367


In [ ]:
# 지도 시각화를 위한 자치구 위치 데이터 적재 및 확인
offices = pd.read_csv("https://raw.githubusercontent.com/logistex/vd21/main/offices.csv", engine='python')
offices.sample(5)

,자치구,자치구청,위도,경도,주소
6,중랑구,중랑구청,37.606560,127.092624,대한민국 서울특별시 중랑구 신내동 662
21,서초구,서초구청,37.483574,127.032661,대한민국 서울특별시 서초구 서초2동 남부순환로 2584
4,광진구,광진구청,37.538533,127.082377,대한민국 서울특별시 광진구 자양1동 자양로 117
3,성동구,성동구청,37.563072,127.036669,대한민국 서울특별시 성동구 행당동 고산자로 270
17,금천구,금천구청,37.451853,126.902036,대한민국 서울특별시 금천구


In [ ]:
# 자치구 경계선에 관한 지도 데이터
seoul_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
seoul = alt.topo_feature(seoul_url, 'seoul_municipalities_geo')
seoul

UrlData({
  format: TopoDataFormat({
    feature: 'seoul_municipalities_geo',
    type: 'topojson'
  }),
  url: 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
})

## 3. 버블차트 시각화

### 3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 

- 함수 `bubble(year, color)`를 정의하라. 
- 매개변수 `year`에 정수 `2015`를 지정하면 2015년 차트를 반환한다. 
- 매개변수 `color`에 색상 이름을 지정하면, 해당 색상으로 버블과 회귀선이 그려진 차트를 반환한다. 


In [ ]:
# 회귀선 버블차트 시각화 함수 정의

def bubble(year, color):   
    bb = alt.Chart(df).mark_circle().transform_calculate(
        CCTV_비율 = 'datum.CCTV_규모_{0} / datum.인구_규모_{0} *100'.format(year)
    ).encode(
        x=alt.X('인구_규모_{0}'.format(year),
               scale = alt.Scale(domain=[100000,700000]),
                title = '인구 규모 (단위:명)'),
        y = alt.Y('CCTV_규모_{0}'.format(year),
                  scale = alt.Scale(domain=[0,7000]),
                 title = 'CCTV 설치 규모 (단위:대)'),
        color = alt.value(color),
        opacity = alt.value(0.5),
        tooltip = [
            alt.Tooltip('자치구:N'),
            alt.Tooltip('CCTV_비율:Q', format='.1%'),
            alt.Tooltip('CCTV_규모_{0}'.format(year)),
            alt.Tooltip('인구_규모_{0}'.format(year)),
        ],
        size = alt.Size('CCTV_비율:Q', scale=alt.Scale(range=[10,1000]), legend=None),
        order = alt.Order('CCTV_비율:Q', sort='descending')
        ).properties(
        width=600,
        height=500)

    reg = bb.transform_regression("인구_규모_{0}".format(year), "CCTV_규모_{0}".format(year)
                                   ).mark_line(opacity=0.5) 

    return alt.layer(
        bb,
        reg
    )  

### 3.2 2015년도 차트 시각화

In [ ]:
# 2015년도 차트를 파란색으로 시각화
bb_reg_2015 = bubble(2015, "blue")
bb_reg_2015

alt.LayerChart(...)

### 3.3 2020년도 차트 시각화

In [ ]:
# 2020년도 차트를 빨간색으로 시각화
bb_reg_2020 = bubble(2020,"red")
bb_reg_2020

alt.LayerChart(...)

### 3.4 이동궤적 시각화

In [ ]:
# 버블 이동궤적 시각화

move = alt.Chart(df).mark_line().transform_calculate(
    증가율= '(datum.CCTV_규모_2020 - datum.CCTV_규모_2015)/datum.CCTV_규모_2015'
).encode(
    x= alt.X('인구_규모_2015:Q', scale = alt.Scale(domain=[100000, 700000])),
    x2= alt.X2('인구_규모_2020:Q'),
    y = alt.Y('CCTV_규모_2015:Q', scale = alt.Scale(domain = [0,7000])),
    y2 = alt.Y2('CCTV_규모_2020:Q'),
    size = alt.Size('증가율:Q',legend = None),
    color = alt.Color('증가율:Q', legend = None),
    tooltip = [
        alt.Tooltip('자치구:N'),
        alt.Tooltip('증가율:Q', format='.1%')
    ]
).properties(
    width = 600,
    height = 400
)

move

alt.Chart(...)

### 3.5 시각화 결과 통합

In [ ]:
# 시각화 결과 통합

layer = alt.layer(
        bb_reg_2015,
        bb_reg_2020,
        move
).resolve_scale(
    color='independent',
    size = 'independent'
).properties(
    title = {
        "text": ["자치구별 인구 대비 CCTV 설치 규모", "(2015년 대 2020년)"],
        "subtitle": ["- 2015년은 푸른색, 2020년은 빨간색 원으로 표시", "- 원 크기는 인구 대비 CCTV 비율", "- 직선 두께와 색상은 2015년 대비 2020년 증가율"]
    }
)
layer

alt.LayerChart(...)

- 버블 차트 시각화 결론 

해당 버블차트는 자치구별 인구 대비 CCTV 설치 규모를 시각화한것입니다. 파란원은 2015년을, 빨간원은 2020년을 나타냅니다. 우선 전체적인 흐름을 보면 5년사이 CCTV 설치 비율이 서울시 내에서 전체적으로 증가했다는 것을 확인할 수 있습니다. 

이동 궤적선의 색과 두께는 2015년 대비 2020년의 증가율을 의미합니다. 이 중 가장 두꺼운 궤적선은 강북구입니다. 강북구는 2015년 대비 CCTV 설치 비율이 가장 많이 증가한 자치구입니다. 증가율은 약 811%입니다.

이동 궤적선이 움직인 방향을 보면, CCTV의 설치 규모는 전체적으로 증가하였지만, 인구 규모는 크게 변화가 없었다는 것을 알 수 있습니다. 대부분의 자치구에서 인구가 조금씩 감소하는 성향을 보였으며, 노원구의 경우 서울시를 통틀어 제일 많이 인구가 감소한 것을 확인할 수 있습니다.
이동궤적선의 길이는 CCTV 설치 규모의 변화 폭입니다. 이 중 가장 변화가 두드러지는 자치구는 강남구입니다. 강남구는 2015년에는 CCTV의 설치규모가 약 1800대였는데, 2020년대는 약 6600대로 크게 증가한 것을 확인할 수 있습니다.

결과적으로 5년동안 인구의 수는 전체적으로 감소했지만, CCTV의 설치 규모는 증가하였다는 사실을 알 수 있습니다.



## 4. 단계구분도 시각화

### 4.1 자치구 위치 시각화

In [ ]:
points = alt.Chart(offices).mark_circle().encode(
    longitude = '경도:Q',
    latitude = '위도:Q',
    size = alt.value(50),
    tooltip = '자치구청'
).properties(
    width = 400, height = 300
)

points

alt.Chart(...)

### 4.2 단계구분도를 반환하는 함수 정의

In [ ]:
def map(rate, pop, cctv, year):
  ch = alt.Chart(seoul).mark_geoshape(
      stroke = '#aaa', strokeWidth=0.25
  ).transform_lookup(
      lookup = 'no',
      from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2015', '인구_규모_2015'])
  ).transform_calculate(
       CCTV_비율 = 'datum.CCTV_규모_2015 / datum.인구_규모_2015 *100'.format(year)
  ).encode(
    color=alt.Color('CCTV_비율:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('CCTV_비율:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_{0}'.format(year, ',d')),
            alt.Tooltip('인구_규모_{0}'.format(year, ',d'))]
).properties(
      title = {
          "text":"CCTV 설치 비율",
          "subtitle":"{0}년".format(year)
      },
      width = 400, height = 300
  )

  return ch

### 4.3 단계구분도 종합

In [ ]:
map2015 = alt.Chart(seoul).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup = 'properties.ESRI_PK',
    from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2015', '인구_규모_2015'])
).transform_calculate(
    cctv_rate=f'(datum.CCTV_규모_2015/datum.인구_규모_2015)*100'
).encode(
    color=alt.Color('cctv_rate:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('cctv_rate:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_2015:Q', format=',d'),
             alt.Tooltip('인구_규모_2015:Q', format=',d')]
).properties(
    title = {
        "text":"CCTV 설치 비율",
        "subtitle":"2015년"
    },
    width = 400, height = 300
)

m2015 = map2015 + points

map2020 = alt.Chart(seoul).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup = 'properties.ESRI_PK',
    from_ = alt.LookupData(data=df, key='no', fields=['자치구', 'CCTV_규모_2020', '인구_규모_2020'])
).transform_calculate(
    cctv_rate=f'(datum.CCTV_규모_2020/datum.인구_규모_2020)*100'
).encode(
    color=alt.Color('cctv_rate:Q',legend=None),
    tooltip=['자치구:N',
             alt.Tooltip('cctv_rate:Q', format='.2%'),
             alt.Tooltip('CCTV_규모_2020:Q', format=',d'),
             alt.Tooltip('인구_규모_2020:Q', format=',d')]
).properties(
    title = {
        "text":"CCTV 설치 비율",
        "subtitle":"2020년"
    },
    width = 400, height = 300
)

m2020 = map2020+points

alt.hconcat(m2015, m2020).configure_view(stroke=None)

alt.HConcatChart(...)

- 단계구분도 시각화 결론

해당 단계구분도는 서울시의 CCTV 설치 비율과 자치구청의 위치를 나타내고 있습니다.  

색이 옅은 녹색에 가까울수록 CCTV의 설치비율이 낮으며, 색이 짙어질수록 CCTV의 설치비율이 높습니다.   2015년과 2020년 두개의 단계구분도를 비교해보면 전체적으로 색이 짙어졌다는 것을 알 수 있습니다. 이는 전체적으로 서울시의 CCTV 설치 비율이 증가했다는 것을 의미합니다. 전체적으로 서울의 중심부에 가까워질수록 색이 짙으며, 외곽쪽으로 갈수록 색이 연해지는 경향을 띄고 있는데, 이는 2015년과 2020년 둘 다 동일한 사항입니다.  

2015년도 서울시에서 CCTV 설치 비율이 가장 높았던 것은 종로구입니다. 그에 반해 5년이 지난 2020년에는 중구의 CCTV 설치 비율이 가장 높다는 것을 확인할 수 있습니다. 

## 5. 결론


같은 데이터를 이용한 두가지의 시각화 결과를 통해 서울시의 CCTV 설치 비율이 전체적으로 증가했다는 사실을 알 수 있습니다.  

다만, 동일한 데이터를 이용했음에도 불구하고 두 시각화 결과에서 얻을 수 있는 정보는 상이했습니다. 버블차트에서는 수치가 있는 차트이기 때문에, 인구 규모와 CCTV 설치 비율간의 상관관계를 한 눈에 확인할 수 있습니다. 또한 회귀선과 이동 궤적선을 통해 전체적인 데이터의 흐름 파악이 가능합니다. 또한  2015년과 2020년 데이터의 컬러 코딩을 다르게 하였기 때문에 데이터의 범주를 쉽게 구분할 수 있었습니다. 

하지만 일반인이 보기에 용이한 데이터 시각화 자료는 지도 시각화를 이용한 단계 구분도라고 생각합니다. 단계 구분도는 수치적인 표현이 없기 때문에 어떤 구가 얼마만큼 바뀌었는지에 대해 명확히 알 수는 없지만, 색의 변화를 통해 전체적으로 CCTV 설치 비율이 늘어났다는 사실을 알 수 있습니다. 또한 지도 시각화를 했기 때문에 해당 시각화 자료가 서울의 자치구를 기준으로 나누었다는 사실을 한 눈에 알 수 있습니다. 그렇기에 어떤 구가 어떻게 변화했는지에 대해 정확히 할 수 있는 것도 단계 구분도라고 생각합니다.  

같은 데이터를 이용해서 데이터를 시각화하더라도 어떤 방식을 이용하느냐에 따라 얻을 수 있는 결과값이 달라진다는 것을 알 수 있었습니다. 이번 과제와 같이 버블차트를 이용하면 핵심 변수가 무엇인지 한 눈에 파악할 수 있다는 장점도 있지만, 원들이 겹치는 부분에서는 정보가 소실될 수도 있습니다. 이와 대조되게 단계구분도를 사용하면 버블이 겹치는 곳이 없기 때문에 정보의 소실은 없지만, 핵심 변수가 무엇인지, 어느정도의 수치가 변화했는지에 대해서는 파악할수가 없었습니다.  
결론적으로 시각화 방법마다 장단점이 명확하기 때문에 시각화 자료를 보는 대상이 누군지에 따라 적절한 방법을 택해야한다고 생각합니다.